<a href="https://colab.research.google.com/github/YusukeTakahashi2001/Pytorch-Tutorial_Des2020/blob/main/ex_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ERROR

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/IPhone-Macbook

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/IPhone-Macbook


In [2]:
!pip install comet_ml
from comet_ml import Experiment

In [3]:
# パッケージのimport
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [4]:
experiment = Experiment(
    api_key="1H4oWzrtQdGj3uKZIDjQomMQs",
    project_name="phone-classification",
    workspace="yusuketakahashi2001",
)


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yusuketakahashi2001/phone-classification/dcf0e1963236455b807cb6dcd7d11aa1



In [5]:
#Create Dataset
from utils.dataloader_image_classification import ImageTransform,make_datapath_list, Dataset

In [7]:
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

# Datasetを作成する
size = 600
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = Dataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = Dataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')


# DataLoaderを作成する
batch_size = hyper_params["batch_size"]

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}



./data/phone_data/train/**/*.jpg
./data/phone_data/val/**/*.jpg


In [8]:
# VGG-16モデルのインスタンスを生成
use_pretrained = True  # 学習済みのパラメータを使用
net = models.vgg16(pretrained=use_pretrained)

# VGG16の最後の出力層の出力ユニットをアリとハチの2つに付け替える
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# 訓練モードに設定
net.train()
print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました')


ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました


In [9]:
# ファインチューニングで学習させるパラメータを、変数params_to_updateの1～3に格納する

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 学習させる層のパラメータ名を指定
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# パラメータごとに各リストに格納する
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1に格納：", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2に格納：", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3に格納：", name)

    else:
        param.requires_grad = False
        print("勾配計算なし。学習しない：", name)

params_to_update_1に格納： features.0.weight
params_to_update_1に格納： features.0.bias
params_to_update_1に格納： features.2.weight
params_to_update_1に格納： features.2.bias
params_to_update_1に格納： features.5.weight
params_to_update_1に格納： features.5.bias
params_to_update_1に格納： features.7.weight
params_to_update_1に格納： features.7.bias
params_to_update_1に格納： features.10.weight
params_to_update_1に格納： features.10.bias
params_to_update_1に格納： features.12.weight
params_to_update_1に格納： features.12.bias
params_to_update_1に格納： features.14.weight
params_to_update_1に格納： features.14.bias
params_to_update_1に格納： features.17.weight
params_to_update_1に格納： features.17.bias
params_to_update_1に格納： features.19.weight
params_to_update_1に格納： features.19.bias
params_to_update_1に格納： features.21.weight
params_to_update_1に格納： features.21.bias
params_to_update_1に格納： features.24.weight
params_to_update_1に格納： features.24.bias
params_to_update_1に格納： features.26.weight
params_to_update_1に格納： features.26.bias
params_to_update_1に格納： f

In [10]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)


In [14]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer):

    # 初期設定
    num_epochs = hyper_params["num_epochs"]
    # GPUが使えるかを確認
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in [ 'train','val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            class_correct = [0, 0]
            class_total = [0, 0]
      
            # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # GPUが使えるならGPUにデータを送る
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # optimizerを初期化
                optimizer.zero_grad()
                             

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    
                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)


                for batch_id in range(labels.shape[0]):
                  
                  target_label = int(labels.data[batch_id])
                  pred_label = int(preds.data[batch_id])
                  if target_label == 0 and target_label == pred_label:
                    class_correct[0] += 1
                    class_total[0] += 1
                  if target_label == 1 and target_label == pred_label:
                    class_correct[1] += 1
                    class_total[1] += 1


                
            

            # epochごとのlossと正解率をcomet_mlに格納
            metrics = {}
            if phase == 'train':
              metrics['train_loss'] = epoch_loss / len(dataloaders_dict[phase].dataset)
              metrics['train_acc']  = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
              metrics['train_normal_acc'] = class_correct[0]/class_total[0] *100
              metrics['train_broken_acc'] = class_correct[1]/class_total[1] *100
          

            elif phase == 'val':
              metrics['test_loss']  =  epoch_loss / len(dataloaders_dict[phase].dataset)
              metrics['test_acc']  = epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
              metrics['test_normal_acc'] = class_correct[0]/class_total[0] *100
              metrics['test_broken_acc'] = class_correct[1]/class_total[1] *100
          
              
              



            experiment.log_metrics(metrics, step=epoch)
            
    experiment.end()
    # PyTorchのネットワークパラメータの保存
    save_path = './saveweight/weights_fine_tuning.pth'
    torch.save(net.state_dict(), save_path)
    for i in range(2):
      print('{}/{} ({:.2f})'.format(class_correct[i], class_total[i], class_correct[i]/class_total[i]*100)) 
    print('Finish')


In [6]:
hyper_params = {
    "batch_size": 4,
    "num_epochs": 5,
}
experiment.log_parameters(hyper_params)

In [ ]:
# 学習・検証を実行する
train_model(net, dataloaders_dict, criterion, optimizer)